In [4]:
from data import Data
import numpy as np
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
d = Data(first=1000)
X_train,y_train = d.get_10()
X_cross, y_cross = d.get_dev()
X_test,y_test = d.get_test()

loaded 100
loaded 200
loaded 300
loaded 400
loaded 500
loaded 600
loaded 700
loaded 800
loaded 900
loaded 1000
loaded 1100
loaded 1200
loaded 1300
loaded 1400
loaded 1500
loaded 1600
loaded 1700
loaded 1800
loaded 1900
loaded 2000


In [6]:
X_train = X_train.transpose(0, 3, 1, 2)
y_train = y_train
print(X_train.shape)
print(y_train.shape)
print(X_cross.shape)
print(y_cross.shape)
print(X_test.shape)
print(y_test.shape)

(2000, 3, 128, 128)
(2000,)
(400, 128, 128, 3)
(400,)
(300, 128, 128, 3)
(300,)


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader
from torch.utils.data import sampler
import torch.nn.functional as F
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable